### Robô da prefeitura de volta redonda

### Desafio

Tinhamos um cadastro de notas fiscais na prefeitura de Volta Redonda que era realizado manualmente, pela Lyssa, responsável pelo faturamento, segundo ela, esses cadastros levavam cerca de 15 minutos cada nota, fazendo um colaborador ficar um dia inteiro fazendo esses cadastros.

A nossa primeira solução foi fazer um robô por mapeamento de mouse, teclado e tela, para auxiliar o setor, o robô levava cerca de 1 minuto e meio por nota e não era a melhor solução possível, já que precisaria de um computador apenas para rodar o código, e a plataforma da prefeitura possibilitava o uso do selenium (biblioteca para automação web)

Então como segunda solução, fizemos o robô por selenium, o robô leva cerca de 30 segundos por nota, e além de ser uma solução muito mais ágil, possibilita o usuário de realizar outras atividades, enquanto o robô roda em backdoor.

### Entrando no Site de Volta Redonda

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import pyautogui

df_senha = pd.read_excel(r'senha.xlsx')
senha = df_senha['senha'][0]


###entrar no site da prefeitura

driver = webdriver.Chrome()
driver.get('https://voltaredonda.simplissweb.com.br/contrib/Inicio')

#fechar o pop-up que brotou
while len(driver.find_elements(By.XPATH, '//*[@id="modalBannerVoltaRedonda"]/div/div/div[1]/button/span[1]')) == 0:
    time.sleep(1)
#após esperar clicar no pop-up
driver.find_element(By.XPATH, '//*[@id="modalBannerVoltaRedonda"]/div/div/div[1]/button/span[1]').click()

#digitar login
driver.find_element(By.ID, 'Login_UserName').send_keys('LOGINHERE')

#digitar senha
driver.find_element(By.ID, 'Login_Password').send_keys(senha)
                                                       
#entrar
driver.find_element(By.CLASS_NAME, 'btn-block').click()

### Cadastrando as Notas

In [ ]:
df_lyssa = pd.read_excel(r'FIXO-VOLTA REDONDA-CEF OESTE SUL RJ- COMP.JANEIRO_.xlsx')


###Emitir nova NFS-E

#esperar aparecer o botão NFS-E
while len(driver.find_elements(By.XPATH, '//*[@id="wrap"]/div[2]/div/div[2]/ul/li[1]/a')) == 0:
    time.sleep(1)

#clicar no botão NFS-E
driver.find_element(By.XPATH, '//*[@id="wrap"]/div[2]/div/div[2]/ul/li[1]/a').click()

#clicar no botão emitir NFS-E
driver.find_element(By.XPATH, '//*[@id="wrap"]/div[2]/div/div[2]/ul/li[1]/div/div/ul[1]/li[2]/a').click()

time.sleep(5)

#mudar para o iframe
iframe = driver.find_element(By.TAG_NAME, 'iframe')
driver.switch_to.frame(iframe)

for i in range(15):
    #nova emissão de NFS-E
    driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_btn_Nova_Nfse').send_keys(Keys.RETURN)

    ###PAGINA 1

    ##cnae
    cnae = str(df_lyssa['CNAE'][i])
    if cnae == '4322302':
        #se o cnae for 4322302
        
        while len(driver.find_elements(By.ID, 'ctl00_ContentPlaceHolder1_ddl_Cnae')) == 0:
            time.sleep(1)
        
        #clicar no cnae
        driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_ddl_Cnae').click()
        
         #escolher o cnae
        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_ddl_Cnae"]/option[5]').click()

        time.sleep(2)
        
        #clicar no enquadramento
        while len(driver.find_elements(By.ID, 'ctl00_ContentPlaceHolder1_ddl_AtividadeCadastro')) == 0:
            time.sleep(1)
            
        time.sleep(3)
        
        driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_ddl_AtividadeCadastro').click()
        
        #escolher o enquadramento
        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_ddl_AtividadeCadastro"]/option[2]').click()
        
        time.sleep(3)
        
        #digitar o texto
        while len(driver.find_elements(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_txt_Discriminacao"]')) == 0:
            time.sleep(1)
        descriminacao = df_lyssa['D1'][i]
        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_txt_Discriminacao"]').send_keys(descriminacao)

        #avançar
        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_btn_Avancar_Tomador"]').click()

        time.sleep(2)

        
        ###PAGINA 2

        time.sleep(2)

        #selecionar o apelido
        driver.find_elements(By.CSS_SELECTOR, 'a.chosen-single')[0].click()
        apelido = str(df_lyssa['APELIDO'][i])
        if len(apelido) == 5:
            driver.find_elements(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_ddl_Apelido_chosen"]/div/div/input')[0].send_keys(f'0{apelido}', Keys.RETURN)

        else:
            driver.find_elements(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_ddl_Apelido_chosen"]/div/div/input')[0].send_keys(apelido, Keys.RETURN)

        time.sleep(3)

        #ir para a próxima página
        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_btn_Avancar"]').click()

        time.sleep(2)
        
        
         ###PAGINA 3

        #botar o valor
        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_txt_Valor_Servico"]').clear()
        time.sleep(2)
        valor = str(df_lyssa['VALOR'][i])
        if valor[4] == '.' and len(valor) == 6:
            driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_txt_Valor_Servico"]').send_keys(f'{valor}0', Keys.RETURN)
        else:
            driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_txt_Valor_Servico"]').send_keys(valor, Keys.RETURN)

        time.sleep(3)

        #apagar a aliquota
        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_txt_Aliquota"]').clear()

        time.sleep(3)

        #botar a aliquota
        aliquota = str(df_lyssa['ALIQUOTA'][i])
        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_txt_Aliquota"]').send_keys(f'{aliquota}0000')

        time.sleep(2)

        ###clicar em avançar
        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_btn_Avanvar_Conclusao"]').click()
        
        time.sleep(2)
        
        #emitir
        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_btn_Salvar"]').click()
        
        time.sleep(3)
        
        #nova emissão
        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_btn_Voltar_Inicial"]').click()
        
        time.sleep(2)
        
        
        
    
    else:
        #se o cnae for 8121400
        
        while len(driver.find_elements(By.ID, 'ctl00_ContentPlaceHolder1_ddl_Cnae')) == 0:
            time.sleep(1)
        
        #clicar no cnae
        driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_ddl_Cnae').click()

        #escolher o valor
        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_ddl_Cnae"]/option[13]').click()

        time.sleep(2)

        #digitar o texto
        while len(driver.find_elements(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_txt_Discriminacao"]')) == 0:
            time.sleep(1)
        descriminacao = df_lyssa['D1'][i]
        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_txt_Discriminacao"]').send_keys(descriminacao)

        #avançar
        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_btn_Avancar_Tomador"]').click()

        time.sleep(2)

        ###PAGINA 2

        time.sleep(2)

        #selecionar o apelido
        driver.find_elements(By.CSS_SELECTOR, 'a.chosen-single')[0].click()
        apelido = str(df_lyssa['APELIDO'][i])
        if len(apelido) == 5:
            driver.find_elements(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_ddl_Apelido_chosen"]/div/div/input')[0].send_keys(f'0{apelido}', Keys.RETURN)

        else:
            driver.find_elements(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_ddl_Apelido_chosen"]/div/div/input')[0].send_keys(apelido, Keys.RETURN)

        time.sleep(3)

        #ir para a próxima página
        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_btn_Avancar"]').click()

        time.sleep(2)

        ###PAGINA 3

        #selecionar a cidade
        driver.find_elements(By.CLASS_NAME, 'chosen-single')[2].click()
        cidade = str(df_lyssa['CIDADE'][i])
        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_ddl_Orgao_Gerador_Es_Municipio_chosen"]/div/div/input').send_keys(cidade, Keys.RETURN)

        time.sleep(5)

        #botar o valor
        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_txt_Valor_Servico"]').clear()
        time.sleep(2)
        valor = str(df_lyssa['VALOR'][i])
        if valor[4] == '.' and len(valor) == 6:
            driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_txt_Valor_Servico"]').send_keys(f'{valor}0', Keys.RETURN)
        else:
            driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_txt_Valor_Servico"]').send_keys(valor, Keys.RETURN)

        time.sleep(3)

        #apagar a aliquota
        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_txt_Aliquota"]').clear()

        time.sleep(3)

        #botar a aliquota
        aliquota = str(df_lyssa['ALIQUOTA'][i])
        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_txt_Aliquota"]').send_keys(f'{aliquota}0000')


        #clicar em sim

        time.sleep(4)

        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_rbl_Retencao_Fonte_0"]').click()

        time.sleep(3)

        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_rbl_Retencao_Fonte_0"]').click()

        time.sleep(2)
        
        #clicar em avançar
        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_btn_Avanvar_Conclusao"]').click()
        
         #emitir
        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_btn_Salvar"]').click()
        
        time.sleep(3)
        
        #nova emissão
        driver.find_element(By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_btn_Voltar_Inicial"]').click()
        
        time.sleep(2)